# Create the Custom Model and host via Provisioned Throughput

## Import libraries

In [ ]:
import boto3
from sagemaker import Session
import uuid  # Import the 'uuid' module for generating a unique identifier

# Create a session using the provided AWS SDK sessions
session = Session(boto_session=boto3.session.Session(),
                sagemaker_client=boto3.client('sagemaker'),
                sagemaker_runtime_client=boto3.client('runtime.sagemaker'))

# Initialize Bedrock client
bedrock = boto3.client(service_name='bedrock')

## Set parameters for the training job

Retrieve the Bedrock's IAM Execution Role and its ARN. 


>If you are an AWS Workshop participant, the role was automatically created for you during the environment's provisioning.

>If you are using your own AWS account, please make sure to execute the code in notebook `00-setup-for-self-paced-users.ipynb`.

In [9]:
import boto3

client = boto3.client(service_name='iam')
response = client.get_role(
    RoleName='Bedrock-Exec-Role-For-Workshop'
)

roleArn = response["Role"]["Arn"]

In [ ]:
# Base model to use
basemodelId = 'amazon.titan-text-express-v1'

# Model ID for provisioned throughput
# https://docs.aws.amazon.com/bedrock/latest/userguide/prov-thru-api.html
baseModelIdentifierForProvisonedThroughput = "arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-text-express-v1:0:8k"
job_prefix = "customTitan"

# Generate a unique identifier for the job and custom model name
job_uuid = str(uuid.uuid4())[:8]  # Extracting the first 8 characters for brevity
jobName = f"{job_prefix}-{job_uuid}"
customModelName = f"{job_prefix}-{job_uuid}"

hyperParameters = {
    "epochCount": "2",
    "batchSize": "1",
    "learningRate": "0.00001",
}

# Retrieve the default bucket name from the session
default_bucket = session.default_bucket()

# Specify the training data configuration using the previously uploaded S3 data
s3_train_data = f"s3://{default_bucket}/PreProcessed/train.jsonl"
trainingDataConfig = {"s3Uri": s3_train_data}

# Specify the output data configuration for the custom model
outputDataConfig = {"s3Uri": f"s3://{default_bucket}/CustomModel/"}

## Trigger the bedrock training job

In [ ]:
# Create a job for model customization
jobIdentifier = bedrock.create_model_customization_job(
    jobName = jobName,
    customizationType = "FINE_TUNING",
    customModelName = customModelName,
    roleArn = roleArn,
    baseModelIdentifier = baseModelIdentifierForProvisonedThroughput,
    hyperParameters = hyperParameters,
    trainingDataConfig = trainingDataConfig,
    outputDataConfig = outputDataConfig
)

# Print the identifier for the created job
print(f"Model customization job created with identifier: {jobIdentifier}")

## Monitor the job till the status is shown as "Completed"

In [ ]:
fine_tune_job = bedrock.get_model_customization_job(jobIdentifier=jobIdentifier['jobArn'])
print(fine_tune_job['status'])
# The job may take more than an hour to complete

## Create provisioned no-commit throughput for the custom model (only run the following once the status of the above job is shown as "Completed")

In [ ]:
customModelId=fine_tune_job['outputModelArn']


provisionedModelName = f"{job_prefix}-provisioned-{job_uuid}"

# Create the provisioned capacity without passing any commitment option
provisionedModelArn = bedrock.create_provisioned_model_throughput(
    modelUnits=1,
    provisionedModelName=provisionedModelName, 
    modelId=customModelId
   )['provisionedModelArn']

## Check the provisoned capacity creation status

In [ ]:
# Get Provisioned model status untill it's completed
provisionedModelStatus = bedrock.get_provisioned_model_throughput(provisionedModelId=provisionedModelArn)
print (provisionedModelStatus['status'])
# The process may take more than an hour to complete

## The following values will be referred in the next two notebooks

In [ ]:
print ( f"provisionedModelArn = {provisionedModelArn}")
print ( f"customModelId = {customModelId}")